        select ga.sustcompanyid                         as entity_id,
               ga.rcp_scenario                          as rcp_scenario,
               ga.year                                  as physical_impact_year,
               null::text                               as data_quality_asset_damage_risk,
               null::text                               as data_quality_high_risk_assets,
               null::text                               as data_quality_productive_capacity_loss,
               ga.overall_asset_damage_risk,
               ga.overall_productive_capacity_loss,
               cnt.high_risk_assets_count,
               coalesce(round((cnt.high_risk_assets_count::numeric / nullif(ga.total_assets_count, 0)) *
                        100, 0),0)                         as overall_high_risk_assets_percentage,
               ga.overall_productive_capacity_loss_days as overall_productive_capacity_loss_days,


               xdi_processing.udf_compute_risk_level(((cume_dist() over (w11)) * 100)::int4,
                                      'global')         as industry_category_asset_damage_risk,
               xdi_processing.udf_compute_risk_level(((cume_dist() over (w22)) * 100)::int4,
                                      'global')         as industry_category_high_risk_assets,
               xdi_processing.udf_compute_risk_level(((cume_dist() over (w33)) * 100)::int4,
                                      'global')         as industry_category_productive_capacity_loss,
               rir.industry_category_regional_infrastructure_risk,
               lcir.industry_category_local_critical_infrastructure_risk,
 
               -- de pus peergroup la groupby
               ((cume_dist() over (w11)) * 100)::int4   as industry_percentile_asset_damage_risk,
               ((cume_dist() over (w22)) * 100)::int4   as industry_percentile_high_risk_assets,
               ((cume_dist() over (w33)) * 100)::int4   as industry_percentile_productive_capacity_loss,
               rir.industry_percentile_regional_infrastructure_risk,
               lcir.industry_percentile_local_critical_infrastructure_risk,

               ga.total_assets_count                    as total_assets_count,
               xdi_processing.udf_compute_risk_level(((cume_dist() over (w1)) * 100)::int4,
                                      'global')         as overall_category_asset_damage_risk,
               xdi_processing.udf_compute_risk_level(((cume_dist() over (w2)) * 100)::int4,
                                      'global')         as overall_category_high_risk_assets,
               xdi_processing.udf_compute_risk_level(((cume_dist() over (w3)) * 100)::int4,
                                      'global')         as overall_category_productive_capacity_loss,
               rir.overall_category_regional_infrastructure_risk,
               lcir.overall_category_local_critical_infrastructure_risk,           
                        
               ((cume_dist() over (w1)) * 100)::int4    as overall_percentile_asset_damage_risk,
               ((cume_dist() over (w2)) * 100)::int4    as overall_percentile_high_risk_assets,
               ((cume_dist() over (w3)) * 100)::int4    as overall_percentile_productive_capacity_loss,
               rir.overall_percentile_regional_infrastructure_risk,
               lcir.overall_percentile_local_critical_infrastructure_risk,
               tc.top_hazard_contributor_for_asset_damage_risk,
			   tc.top_country_contributor_for_asset_damage_risk,
			   tc.top_hazard_contributor_for_productive_capacity_loss,
			   tc.top_country_contributor_for_productive_capacity_loss,
			   tc.top_country_contributor_for_high_risk_assets,
			   tc.top_country_contributor_for_regional_risk,
			   tc.top_country_contributor_for_local_critical_infrastructure

In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
# # MERGE ALL CSV INTO ONE

# # Specify the path to the CSV files
# path = "C:/Users/jkumava/Downloads/PCRM/global/"  # Adjust this to your CSV files' directory
# all_files = glob.glob(path + "*.csv")

# dfs = []

# for filename in all_files:
#     df = pd.read_csv(filename)
#     dfs.append(df)

# merged_df = pd.concat(dfs, ignore_index=True)

# merged_df.to_csv("C:/Users/jkumava/Downloads/PCRM/global/merged_file.csv", index=False)


In [3]:
global_df = pd.read_csv('global_risk.csv')

In [4]:
global_df = global_df[global_df['fl_deleted'] != True].sort_values(by = ['entity_id',	'rcp_scenario',	'physical_impact_year']).reset_index(drop=True)
global_df.columns

Index(['entity_id', 'rcp_scenario', 'physical_impact_year',
       'data_quality_asset_damage_risk', 'data_quality_high_risk_assets',
       'data_quality_productive_capacity_loss', 'high_risk_assets_count',
       'industry_category_asset_damage_risk',
       'industry_category_high_risk_assets',
       'industry_category_productive_capacity_loss',
       'industry_percentile_asset_damage_risk',
       'industry_percentile_high_risk_assets',
       'industry_percentile_productive_capacity_loss',
       'overall_asset_damage_risk', 'overall_category_asset_damage_risk',
       'overall_category_high_risk_assets',
       'overall_high_risk_assets_percentage',
       'overall_percentile_asset_damage_risk',
       'overall_percentile_high_risk_assets',
       'overall_percentile_productive_capacity_loss',
       'overall_productive_capacity_loss',
       'overall_productive_capacity_loss_days', 'total_assets_count',
       'dt_created', 'dt_updated', 'fl_deleted', 'timestamps',
       'ove

In [5]:
raw_global_data = global_df[['entity_id',
 'rcp_scenario', 'physical_impact_year',
 'high_risk_assets_count',
 'overall_asset_damage_risk',
 'overall_high_risk_assets_percentage',
 'overall_productive_capacity_loss',
 'overall_productive_capacity_loss_days',
 'top_country_contributor_for_asset_damage_risk',
 'top_country_contributor_for_high_risk_assets',
 'top_country_contributor_for_local_critical_infrastructure',
 'top_country_contributor_for_productive_capacity_loss',
 'top_country_contributor_for_regional_risk',
 'top_hazard_contributor_for_asset_damage_risk',
 'top_hazard_contributor_for_productive_capacity_loss']].copy()
raw_global_data

,entity_id,rcp_scenario,physical_impact_year,high_risk_assets_count,overall_asset_damage_risk,overall_high_risk_assets_percentage,overall_productive_capacity_loss,overall_productive_capacity_loss_days,top_country_contributor_for_asset_damage_risk,top_country_contributor_for_high_risk_assets,top_country_contributor_for_local_critical_infrastructure,top_country_contributor_for_productive_capacity_loss,top_country_contributor_for_regional_risk,top_hazard_contributor_for_asset_damage_risk,top_hazard_contributor_for_productive_capacity_loss
0,1007896757,RCP2.6,2023,67,0.000896,4.0,0.000364,NaN,FRA,FRA,FRA,IND,FRA,Flooding,Extreme Heat
1,1007896757,RCP2.6,2028,67,0.000956,4.0,0.000394,NaN,FRA,FRA,FRA,IND,FRA,Flooding,Extreme Heat
2,1007896757,RCP2.6,2030,67,0.000958,4.0,0.000401,NaN,FRA,FRA,FRA,IND,FRA,Flooding,Extreme Heat
3,1007896757,RCP2.6,2050,68,0.001481,4.0,0.000643,NaN,FRA,FRA,FRA,FRA,FRA,Coastal Inundation,Extreme Heat
4,1007896757,RCP2.6,2100,85,0.003231,5.0,0.001338,NaN,FRA,FRA,FRA,FRA,FRA,Coastal Inundation,Extreme Heat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55780,2014520133,RCP2.6,2023,1,0.000763,4.0,0.000329,NaN,BRA,BRA,BRA,BRA,BRA,Flooding,Extreme Heat
55781,2014520133,RCP2.6,2028,1,0.000765,4.0,0.000347,NaN,BRA,BRA,BRA,BRA,BRA,Flooding,Extreme Heat
55782,2014520133,RCP2.6,2030,1,0.000766,4.0,0.000353,NaN,BRA,BRA,BRA,BRA,BRA,Flooding,Extreme Heat
55783,2014520133,RCP2.6,2050,1,0.000775,4.0,0.000406,NaN,BRA,BRA,BRA,BRA,BRA,Flooding,Extreme Heat


In [6]:
rei_overall_df = pd.read_csv("C:/Users/jkumava/Downloads/PCRM/regional_overall.csv")
rei_overall_df = rei_overall_df[rei_overall_df['scenario_code'] == "RCP2.6"].reset_index(drop=True)
rei_overall_df

,entity_id,scenario_code,year,clientindustryid,overall_regional_infrastructure_risk
0,1007896757,RCP2.6,2023,402047,0.002150
1,1007896757,RCP2.6,2028,402047,0.002203
2,1007896757,RCP2.6,2030,402047,0.002231
3,1007896757,RCP2.6,2050,402047,0.002543
4,1007896757,RCP2.6,2100,402047,0.003382
...,...,...,...,...,...
55780,2014520133,RCP2.6,2023,151019,0.001718
55781,2014520133,RCP2.6,2028,151019,0.001781
55782,2014520133,RCP2.6,2030,151019,0.001814
55783,2014520133,RCP2.6,2050,151019,0.002221


In [7]:
rei_overall_df = rei_overall_df.rename(columns = {'scenario_code':'rcp_scenario' , 'year': 'physical_impact_year' } )
rei_overall_df

,entity_id,rcp_scenario,physical_impact_year,clientindustryid,overall_regional_infrastructure_risk
0,1007896757,RCP2.6,2023,402047,0.002150
1,1007896757,RCP2.6,2028,402047,0.002203
2,1007896757,RCP2.6,2030,402047,0.002231
3,1007896757,RCP2.6,2050,402047,0.002543
4,1007896757,RCP2.6,2100,402047,0.003382
...,...,...,...,...,...
55780,2014520133,RCP2.6,2023,151019,0.001718
55781,2014520133,RCP2.6,2028,151019,0.001781
55782,2014520133,RCP2.6,2030,151019,0.001814
55783,2014520133,RCP2.6,2050,151019,0.002221


In [8]:
fmi_overall_df = pd.read_csv("C:/Users/jkumava/Downloads/PCRM/fmi_overall.csv")
fmi_overall_df.head()

,entity_id,scenario_code,year,clientindustryid,overall_local_critical_infrastructure_risk
0,1007896757,RCP2.6,2023,402047,0.000165
1,1007896757,RCP2.6,2028,402047,0.000175
2,1007896757,RCP2.6,2030,402047,0.000179
3,1007896757,RCP2.6,2050,402047,0.000268
4,1007896757,RCP2.6,2100,402047,0.000474


In [9]:
fmi_overall_df = fmi_overall_df[fmi_overall_df['scenario_code'] == "RCP2.6"].reset_index(drop=True)
fmi_overall_df = fmi_overall_df.rename(columns = {'scenario_code':'rcp_scenario' , 'year': 'physical_impact_year' } )
fmi_overall_df

,entity_id,rcp_scenario,physical_impact_year,clientindustryid,overall_local_critical_infrastructure_risk
0,1007896757,RCP2.6,2023,402047,0.000165
1,1007896757,RCP2.6,2028,402047,0.000175
2,1007896757,RCP2.6,2030,402047,0.000179
3,1007896757,RCP2.6,2050,402047,0.000268
4,1007896757,RCP2.6,2100,402047,0.000474
...,...,...,...,...,...
55765,2014520133,RCP2.6,2023,151019,0.000277
55766,2014520133,RCP2.6,2028,151019,0.000294
55767,2014520133,RCP2.6,2030,151019,0.000301
55768,2014520133,RCP2.6,2050,151019,0.000441


In [10]:
tzezro_df = pd.read_csv('tzero_202407221801.csv')
tzezro_df = tzezro_df.groupby(['entity_id' , 'scenario_code'], as_index=False).agg( total_assets_count = ('count', 'sum')) [['entity_id', 'total_assets_count']]
tzezro_df = tzezro_df.groupby('entity_id', as_index=False)['total_assets_count'].first()
tzezro_df

,entity_id,total_assets_count
0,1007896757,1871
1,1007896773,2395
2,1007896917,286
3,1007896957,471
4,1007896995,1472
...,...,...
11152,2014218235,262
11153,2014294180,20
11154,2014391261,212
11155,2014391687,34


In [11]:
raw_global_data = raw_global_data.merge(fmi_overall_df, on = ['entity_id', 'rcp_scenario', 'physical_impact_year'], how = 'left' ).merge(rei_overall_df,on = ['entity_id', 'rcp_scenario', 'physical_impact_year','clientindustryid'], how = 'left')\
.merge(tzezro_df, on = 'entity_id' , how='left')

In [12]:
raw_global_data[raw_global_data['high_risk_assets_count'] != global_df['high_risk_assets_count']]

,entity_id,rcp_scenario,physical_impact_year,high_risk_assets_count,overall_asset_damage_risk,overall_high_risk_assets_percentage,overall_productive_capacity_loss,overall_productive_capacity_loss_days,top_country_contributor_for_asset_damage_risk,top_country_contributor_for_high_risk_assets,top_country_contributor_for_local_critical_infrastructure,top_country_contributor_for_productive_capacity_loss,top_country_contributor_for_regional_risk,top_hazard_contributor_for_asset_damage_risk,top_hazard_contributor_for_productive_capacity_loss,clientindustryid,overall_local_critical_infrastructure_risk,overall_regional_infrastructure_risk,total_assets_count


In [14]:
global_df['overall_percentile_high_risk_assets']

0        45
1        44
2        44
3        41
4        36
         ..
55780    52
55781    52
55782    52
55783    48
55784    29
Name: overall_percentile_high_risk_assets, Length: 55785, dtype: int64

In [16]:
# raw_global_data.columns
raw_global_data['overall_high_risk_assets_unrounded'] = (raw_global_data['high_risk_assets_count']/(raw_global_data['total_assets_count']) * 100)
# raw_global_data.groupby(['rcp_scenario', 'physical_impact_year'])['overall_high_risk_assets_unrounded'].rank(pct=True) * 100


## Tested Columns

### overall_asset_damage_risk  , high_risk_assets_count, overall_percentile_local_critical_infrastructure_risk ,industry_percentile_local_critical_infrastructure_risk

### Not Matched : - overall_percentile_high_risk_assets : 2 rows, industry_percentile_high_risk_assets , industry_percentile_high_risk_assets:296 , industry_percentile_productive_capacity_loss:326, overall_percentile_regional_infrastructure_risk:588,. industry_percentile_regional_infrastructure_risk :600

In [44]:
raw_global_data[raw_global_data['industry_percentile_local_critical_infrastructure_risk'] != global_df['industry_percentile_local_critical_infrastructure_risk']]['industry_percentile_local_critical_infrastructure_risk']

11365   NaN
11366   NaN
11367   NaN
11368   NaN
11369   NaN
41315   NaN
41316   NaN
41317   NaN
41318   NaN
41319   NaN
42190   NaN
42191   NaN
42192   NaN
42193   NaN
42194   NaN
Name: industry_percentile_local_critical_infrastructure_risk, dtype: float64

In [43]:
global_df[raw_global_data['industry_percentile_regional_infrastructure_risk'] != global_df['industry_percentile_regional_infrastructure_risk']]['industry_percentile_regional_infrastructure_risk']

27       43
170      23
1848     61
2016     43
2052     79
         ..
55514    52
55556    41
55560    62
55636    63
55668    87
Name: industry_percentile_regional_infrastructure_risk, Length: 600, dtype: int64

In [58]:
raw_global_data['overall_percentile_asset_damage_risk'] = (raw_global_data.groupby(['rcp_scenario', 'physical_impact_year'])['overall_asset_damage_risk'].rank(pct=True,  method='max') * 100).round()
raw_global_data['overall_percentile_high_risk_assets'] = (raw_global_data.groupby(['rcp_scenario', 'physical_impact_year'])['overall_high_risk_assets_unrounded'].rank(pct=True, method='max') * 100).round()
raw_global_data['overall_percentile_productive_capacity_loss'] = (raw_global_data.groupby(['rcp_scenario', 'physical_impact_year'])['overall_productive_capacity_loss'].rank(pct=True, method='max') * 100).round()

raw_global_data['industry_percentile_asset_damage_risk'] = (raw_global_data.groupby(['rcp_scenario', 'physical_impact_year','clientindustryid'])['overall_asset_damage_risk'].rank(pct=True, method='max') * 100).round()
raw_global_data['industry_percentile_high_risk_assets'] = (raw_global_data.groupby(['rcp_scenario', 'physical_impact_year','clientindustryid'])['overall_high_risk_assets_unrounded'].rank(pct=True, method='max') * 100).round()
raw_global_data['industry_percentile_productive_capacity_loss'] = (raw_global_data.groupby(['rcp_scenario', 'physical_impact_year','clientindustryid'])['overall_productive_capacity_loss'].rank(pct=True, method='max') * 100).round()

In [59]:
raw_global_data['overall_percentile_regional_infrastructure_risk'] = (raw_global_data.groupby(['rcp_scenario', 'physical_impact_year'])['overall_regional_infrastructure_risk'].rank(pct=True, method='max') * 100).round()
raw_global_data['overall_percentile_local_critical_infrastructure_risk'] = (raw_global_data.groupby(['rcp_scenario', 'physical_impact_year'])['overall_local_critical_infrastructure_risk'].rank(pct=True, method='max') * 100).round()

raw_global_data['industry_percentile_regional_infrastructure_risk'] = (raw_global_data.groupby(['rcp_scenario', 'physical_impact_year','clientindustryid'])['overall_regional_infrastructure_risk'].rank(pct=True, method='max') * 100).round()
raw_global_data['industry_percentile_local_critical_infrastructure_risk'] = (raw_global_data.groupby(['rcp_scenario', 'physical_impact_year','clientindustryid'])['overall_local_critical_infrastructure_risk'].rank(pct=True, method='max') * 100).round()

In [60]:
tier_conditions =  pd.read_csv('tier_conditions.csv')

def categorize_level(level, tier_conditions,tier_type):
    if pd.isna(level):
        return np.nan
    tier_country = tier_conditions[tier_conditions['tier_type'] == tier_type].iloc[:,:-1].reset_index(drop=True)
    
    for _, row in tier_country.iterrows():
        if level is None:
            return np.nan
            
        if row['lowerbound'] < level <= row['upperbound']:
            return row['tier']
    return np.nan


def categorize_level1(df,tier_type):

    dct = tier_conditions[tier_conditions['tier_type']=='global'].sort_values(by ='lowerbound' ).to_dict(orient='list')

    # Create bins and labels for pd.cut
    bins = sorted(set(dct['lowerbound'] + dct['upperbound']))
    labels = dct['tier']
    
    for _, row in tier_country.iterrows():
        if level is None:
            return np.nan
            
        if row['lowerbound'] < level <= row['upperbound']:
            return row['tier']
    return np.nan


In [61]:
dct_global = tier_conditions[tier_conditions['tier_type']=='global'].sort_values(by ='lowerbound' ).to_dict(orient='list')

# Create bins and labels for pd.cut
global_bins = sorted(set(dct_global['lowerbound'] + dct_global['upperbound']))
global_labels = dct_global['tier']

In [62]:
# Conventional Methods

# raw_global_data['overall_category_asset_damage_risk'] = raw_global_data['overall_percentile_asset_damage_risk'].apply(lambda x: categorize_level(x,tier_conditions, 'global'))
# raw_global_data['overall_category_high_risk_assets'] = raw_global_data['overall_percentile_high_risk_assets'].apply(lambda x: categorize_level(x,tier_conditions, 'global'))
# raw_global_data['overall_category_productive_capacity_loss'] = raw_global_data['overall_percentile_productive_capacity_loss'].apply(lambda x: categorize_level(x,tier_conditions, 'global'))

# #Industry percentile

# raw_global_data['industry_category_asset_damage_risk'] = raw_global_data['industry_percentile_asset_damage_risk'].apply(lambda x: categorize_level(x,tier_conditions, 'global'))
# raw_global_data['industry_category_high_risk_assets'] = raw_global_data['industry_percentile_high_risk_assets'].apply(lambda x: categorize_level(x,tier_conditions, 'global'))
# raw_global_data['industry_category_productive_capacity_loss'] = raw_global_data['industry_percentile_productive_capacity_loss'].apply(lambda x: categorize_level(x,tier_conditions, 'global'))

In [63]:
raw_global_data['overall_category_asset_damage_risk'] = pd.cut(
    raw_global_data['overall_percentile_asset_damage_risk'],
    bins=global_bins,
    labels=global_labels,
    right=True)

raw_global_data['overall_category_high_risk_assets'] = pd.cut(
    raw_global_data['overall_percentile_high_risk_assets'],
    bins=global_bins,
    labels=global_labels,
    right=True)

raw_global_data['overall_category_productive_capacity_loss'] = pd.cut(
    raw_global_data['overall_percentile_productive_capacity_loss'],
    bins=global_bins,
    labels=global_labels,
    right=True)

raw_global_data['overall_category_regional_infrastructure_risk'] = pd.cut(
    raw_global_data['overall_percentile_regional_infrastructure_risk'],
    bins=global_bins,
    labels=global_labels,
    right=True)

raw_global_data['overall_category_local_critical_infrastructure_risk'] = pd.cut(
    raw_global_data['overall_percentile_local_critical_infrastructure_risk'],
    bins=global_bins,
    labels=global_labels,
    right=True)

#Industry percentile

raw_global_data['industry_category_asset_damage_risk'] = pd.cut(
    raw_global_data['industry_percentile_asset_damage_risk'],
    bins=global_bins,
    labels=global_labels,
    right=True)

raw_global_data['industry_category_high_risk_assets'] = pd.cut(
    raw_global_data['industry_percentile_high_risk_assets'],
    bins=global_bins,
    labels=global_labels,
    right=True)

raw_global_data['industry_category_productive_capacity_loss'] = pd.cut(
    raw_global_data['industry_percentile_productive_capacity_loss'],
    bins=global_bins,
    labels=global_labels,
    right=True)

raw_global_data['industry_category_regional_infrastructure_risk'] = pd.cut(
    raw_global_data['industry_percentile_regional_infrastructure_risk'],
    bins=global_bins,
    labels=global_labels,
    right=True)

raw_global_data['industry_category_local_critical_infrastructure_risk'] = pd.cut(
    raw_global_data['industry_percentile_local_critical_infrastructure_risk'],
    bins=global_bins,
    labels=global_labels,
    right=True)

In [64]:
# raw_global_data['overall_category_regional_infrastructure_risk'] = raw_global_data['overall_percentile_regional_infrastructure_risk'].apply(lambda x: categorize_level(x,tier_conditions, 'global'))
# raw_global_data['overall_category_local_critical_infrastructure_risk'] = raw_global_data['overall_percentile_local_critical_infrastructure_risk'].apply(lambda x: categorize_level(x,tier_conditions, 'global'))

# #Industry percentile

# raw_global_data['industry_category_regional_infrastructure_risk'] = raw_global_data['industry_percentile_regional_infrastructure_risk'].apply(lambda x: categorize_level(x,tier_conditions, 'global'))
# raw_global_data['industry_category_local_critical_infrastructure_risk'] = raw_global_data['industry_percentile_local_critical_infrastructure_risk'].apply(lambda x: categorize_level(x,tier_conditions, 'global'))

In [65]:
raw_global_data[raw_global_data['entity_id'] == 1007896757][['entity_id','physical_impact_year','overall_percentile_asset_damage_risk','overall_category_asset_damage_risk']]

,entity_id,physical_impact_year,overall_percentile_asset_damage_risk,overall_category_asset_damage_risk
0,1007896757,2023,26.0,Below Average
1,1007896757,2028,27.0,Below Average
2,1007896757,2030,27.0,Below Average
3,1007896757,2050,40.0,Below Average
4,1007896757,2100,48.0,Average


In [66]:
# raw_global_data[['overall_percentile_regional_infrastructure_risk', 'overall_category_regional_infrastructure_risk','overall_category_regional_infrastructure_risk_cut_test']]

In [67]:
# raw_global_data[raw_global_data['overall_category_regional_infrastructure_risk']!=raw_global_data['overall_category_regional_infrastructure_risk_cut_test']][['overall_percentile_regional_infrastructure_risk', 'overall_category_regional_infrastructure_risk','overall_category_regional_infrastructure_risk_cut_test']]

In [68]:
raw_global_data.drop([col for col in raw_global_data.columns if '_test' in col], axis = 1,inplace=True)

In [69]:
raw_global_data[[
 'industry_percentile_asset_damage_risk',
 'industry_percentile_high_risk_assets',
 'industry_percentile_local_critical_infrastructure_risk',
 'industry_percentile_productive_capacity_loss',
 'industry_percentile_regional_infrastructure_risk',
 'overall_percentile_asset_damage_risk',
 'overall_percentile_high_risk_assets',
 'overall_percentile_local_critical_infrastructure_risk',
 'overall_percentile_productive_capacity_loss',
 'overall_percentile_regional_infrastructure_risk']] = raw_global_data[[
 'industry_percentile_asset_damage_risk',
 'industry_percentile_high_risk_assets',
 'industry_percentile_local_critical_infrastructure_risk',
 'industry_percentile_productive_capacity_loss',
 'industry_percentile_regional_infrastructure_risk',
 'overall_percentile_asset_damage_risk',
 'overall_percentile_high_risk_assets',
 'overall_percentile_local_critical_infrastructure_risk',
 'overall_percentile_productive_capacity_loss',
 'overall_percentile_regional_infrastructure_risk']].round()\
.astype('Int64')

In [116]:
def percentage_calculater_df(df1,df2):
    
    df3 = df1.copy()
    numeric_columns = df1.select_dtypes(include=['number']).columns
    for col in numeric_columns[1:]:
        df3[col] = abs((df1[col]-df2[col])/df1[col])*100

    object_columns = df1.select_dtypes(include=['object']).columns
    
    # Compare object columns and store the result in df3
    for col in object_columns:
        df3[col] = df1[col] == df2[col]

    return df3

def df_comparer_numerical(df3, limit):
    numeric_columns = df3.select_dtypes(include=['number']).columns
    # cols_with_large_values = (df3.loc[:, numeric_columns[1:]] >= limit).any(axis=0)
    # return df3[(df3[numeric_columns[1:]] >= limit).any(axis=1)]
    rows_with_large_values = (df3[numeric_columns[1:]] >= limit).any(axis=1)
    # cols_with_large_values = (df3.loc[:, numeric_columns[1:]] >= limit).any(axis=0)
    cols_with_large_values = (df3.loc[:, numeric_columns[1:]] >= limit).any(axis=0).reset_index()
    cols_with_large_values = cols_with_large_values[cols_with_large_values[0] == True]['index'].tolist()
    # return df3[(df3[numeric_columns[1:]] >= limit).any(axis=1)]
    return df3.loc[rows_with_large_values, ['entity_id']+cols_with_large_values]

def df_comparer_object(df3):
    object_columns = df3.select_dtypes(include=['object']).columns
    rows_with_false_values = (df3[object_columns] == False).any(axis=1)
    # cols_with_large_values = (df3.loc[:, numeric_columns[1:]] >= limit).any(axis=0)
    cols_with_false_values = (df3.loc[:,object_columns] ==False).any(axis=0).reset_index()
    cols_with_false_values = cols_with_false_values[cols_with_false_values[0] == True]['index'].tolist()
    # return df3[(df3[numeric_columns[1:]] >= limit).any(axis=1)]
    return df3.loc[rows_with_false_values, ['entity_id']+cols_with_false_values]

In [74]:
db_global = global_df[[item for item in raw_global_data.columns if item not in ['clientindustryid', 'overall_local_critical_infrastructure_risk','overall_regional_infrastructure_risk','overall_high_risk_assets_unrounded','high_risk_assets_percentage_unrounded']]].sort_values(by=['entity_id','rcp_scenario','physical_impact_year']).reset_index(drop=True).copy()
db_global

,entity_id,rcp_scenario,physical_impact_year,high_risk_assets_count,overall_asset_damage_risk,overall_high_risk_assets_percentage,overall_productive_capacity_loss,overall_productive_capacity_loss_days,top_country_contributor_for_asset_damage_risk,top_country_contributor_for_high_risk_assets,...,overall_category_asset_damage_risk,overall_category_high_risk_assets,overall_category_productive_capacity_loss,overall_category_regional_infrastructure_risk,overall_category_local_critical_infrastructure_risk,industry_category_asset_damage_risk,industry_category_high_risk_assets,industry_category_productive_capacity_loss,industry_category_regional_infrastructure_risk,industry_category_local_critical_infrastructure_risk
0,1007896757,RCP2.6,2023,67,0.000896,4.0,0.000364,NaN,FRA,FRA,...,Below Average,Average,Significantly Below Average,Below Average,Below Average,Below Average,Average,Significantly Below Average,Below Average,Below Average
1,1007896757,RCP2.6,2028,67,0.000956,4.0,0.000394,NaN,FRA,FRA,...,Below Average,Average,Significantly Below Average,Below Average,Below Average,Below Average,Average,Significantly Below Average,Below Average,Below Average
2,1007896757,RCP2.6,2030,67,0.000958,4.0,0.000401,NaN,FRA,FRA,...,Below Average,Average,Significantly Below Average,Below Average,Below Average,Below Average,Average,Significantly Below Average,Below Average,Below Average
3,1007896757,RCP2.6,2050,68,0.001481,4.0,0.000643,NaN,FRA,FRA,...,Below Average,Average,Significantly Below Average,Below Average,Below Average,Average,Average,Significantly Below Average,Below Average,Below Average
4,1007896757,RCP2.6,2100,85,0.003231,5.0,0.001338,NaN,FRA,FRA,...,Average,Below Average,Below Average,Significantly Below Average,Below Average,Average,Average,Below Average,Significantly Below Average,Below Average
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55780,2014520133,RCP2.6,2023,1,0.000763,4.0,0.000329,NaN,BRA,BRA,...,Significantly Below Average,Average,Significantly Below Average,Significantly Below Average,Above Average,Below Average,Average,Significantly Below Average,Significantly Below Average,Above Average
55781,2014520133,RCP2.6,2028,1,0.000765,4.0,0.000347,NaN,BRA,BRA,...,Significantly Below Average,Average,Significantly Below Average,Significantly Below Average,Above Average,Below Average,Average,Significantly Below Average,Significantly Below Average,Above Average
55782,2014520133,RCP2.6,2030,1,0.000766,4.0,0.000353,NaN,BRA,BRA,...,Significantly Below Average,Average,Significantly Below Average,Significantly Below Average,Above Average,Below Average,Average,Significantly Below Average,Significantly Below Average,Above Average
55783,2014520133,RCP2.6,2050,1,0.000775,4.0,0.000406,NaN,BRA,BRA,...,Significantly Below Average,Average,Significantly Below Average,Significantly Below Average,Above Average,Below Average,Average,Significantly Below Average,Below Average,Above Average


In [75]:
df2 = raw_global_data[db_global.columns].sort_values(by=['entity_id','rcp_scenario','physical_impact_year']).reset_index(drop=True).copy()
df2

,entity_id,rcp_scenario,physical_impact_year,high_risk_assets_count,overall_asset_damage_risk,overall_high_risk_assets_percentage,overall_productive_capacity_loss,overall_productive_capacity_loss_days,top_country_contributor_for_asset_damage_risk,top_country_contributor_for_high_risk_assets,...,overall_category_asset_damage_risk,overall_category_high_risk_assets,overall_category_productive_capacity_loss,overall_category_regional_infrastructure_risk,overall_category_local_critical_infrastructure_risk,industry_category_asset_damage_risk,industry_category_high_risk_assets,industry_category_productive_capacity_loss,industry_category_regional_infrastructure_risk,industry_category_local_critical_infrastructure_risk
0,1007896757,RCP2.6,2023,67,0.000896,4.0,0.000364,NaN,FRA,FRA,...,Below Average,Average,Significantly Below Average,Below Average,Below Average,Below Average,Average,Significantly Below Average,Below Average,Below Average
1,1007896757,RCP2.6,2028,67,0.000956,4.0,0.000394,NaN,FRA,FRA,...,Below Average,Average,Significantly Below Average,Below Average,Below Average,Below Average,Average,Significantly Below Average,Below Average,Below Average
2,1007896757,RCP2.6,2030,67,0.000958,4.0,0.000401,NaN,FRA,FRA,...,Below Average,Average,Significantly Below Average,Below Average,Below Average,Below Average,Average,Significantly Below Average,Below Average,Below Average
3,1007896757,RCP2.6,2050,68,0.001481,4.0,0.000643,NaN,FRA,FRA,...,Below Average,Average,Significantly Below Average,Below Average,Below Average,Average,Average,Significantly Below Average,Below Average,Below Average
4,1007896757,RCP2.6,2100,85,0.003231,5.0,0.001338,NaN,FRA,FRA,...,Average,Below Average,Below Average,Significantly Below Average,Below Average,Average,Average,Below Average,Significantly Below Average,Below Average
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55780,2014520133,RCP2.6,2023,1,0.000763,4.0,0.000329,NaN,BRA,BRA,...,Significantly Below Average,Average,Significantly Below Average,Significantly Below Average,Above Average,Below Average,Average,Significantly Below Average,Significantly Below Average,Above Average
55781,2014520133,RCP2.6,2028,1,0.000765,4.0,0.000347,NaN,BRA,BRA,...,Significantly Below Average,Average,Significantly Below Average,Significantly Below Average,Above Average,Below Average,Average,Significantly Below Average,Significantly Below Average,Above Average
55782,2014520133,RCP2.6,2030,1,0.000766,4.0,0.000353,NaN,BRA,BRA,...,Significantly Below Average,Average,Significantly Below Average,Significantly Below Average,Above Average,Below Average,Average,Significantly Below Average,Significantly Below Average,Above Average
55783,2014520133,RCP2.6,2050,1,0.000775,4.0,0.000406,NaN,BRA,BRA,...,Significantly Below Average,Average,Significantly Below Average,Significantly Below Average,Above Average,Below Average,Average,Significantly Below Average,Below Average,Above Average


In [109]:
df_error = percentage_calculater_df(db_global,df2)
df_error

,entity_id,rcp_scenario,physical_impact_year,high_risk_assets_count,overall_asset_damage_risk,overall_high_risk_assets_percentage,overall_productive_capacity_loss,overall_productive_capacity_loss_days,top_country_contributor_for_asset_damage_risk,top_country_contributor_for_high_risk_assets,...,overall_category_asset_damage_risk,overall_category_high_risk_assets,overall_category_productive_capacity_loss,overall_category_regional_infrastructure_risk,overall_category_local_critical_infrastructure_risk,industry_category_asset_damage_risk,industry_category_high_risk_assets,industry_category_productive_capacity_loss,industry_category_regional_infrastructure_risk,industry_category_local_critical_infrastructure_risk
0,1007896757,True,0.0,0.0,0.0,0.0,0.0,NaN,True,True,...,True,True,True,True,True,True,True,True,True,True
1,1007896757,True,0.0,0.0,0.0,0.0,0.0,NaN,True,True,...,True,True,True,True,True,True,True,True,True,True
2,1007896757,True,0.0,0.0,0.0,0.0,0.0,NaN,True,True,...,True,True,True,True,True,True,True,True,True,True
3,1007896757,True,0.0,0.0,0.0,0.0,0.0,NaN,True,True,...,True,True,True,True,True,True,True,True,True,True
4,1007896757,True,0.0,0.0,0.0,0.0,0.0,NaN,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55780,2014520133,True,0.0,0.0,0.0,0.0,0.0,NaN,True,True,...,True,True,True,True,True,True,True,True,True,True
55781,2014520133,True,0.0,0.0,0.0,0.0,0.0,NaN,True,True,...,True,True,True,True,True,True,True,True,True,True
55782,2014520133,True,0.0,0.0,0.0,0.0,0.0,NaN,True,True,...,True,True,True,True,True,True,True,True,True,True
55783,2014520133,True,0.0,0.0,0.0,0.0,0.0,NaN,True,True,...,True,True,True,True,True,True,True,True,True,True


In [110]:
df_comparer_numerical(df_error,15)

,entity_id,overall_percentile_regional_infrastructure_risk
7328,1008257001,25.0
13817,1008755163,25.0
19426,1009416219,25.0
20375,1010747923,25.0
22064,1012387346,25.0


In [113]:
df_comparer_object(df_error)

,entity_id


In [126]:
object_columns = df_error.select_dtypes(include=['object']).columns
rows_with_false_values = (df_error[object_columns] == False).any(axis=1)
# df_error[rows_with_false_values]

cols_with_false_values = (df_error.loc[:,object_columns] ==False).any(axis=0).reset_index()

cols_with_false_values = cols_with_false_values[cols_with_false_values[0] == True]['index'].tolist()
cols_with_false_values
df_error.loc[rows_with_false_values, ['entity_id']+cols_with_false_values]

,entity_id


In [92]:
df2.loc[7328]#,'overall_percentile_regional_infrastructure_risk']

entity_id                                                                     1008257001
rcp_scenario                                                                      RCP2.6
physical_impact_year                                                                2050
high_risk_assets_count                                                                10
overall_asset_damage_risk                                                       0.000601
overall_high_risk_assets_percentage                                                  2.0
overall_productive_capacity_loss                                                0.000269
overall_productive_capacity_loss_days                                                NaN
top_country_contributor_for_asset_damage_risk                                        CAN
top_country_contributor_for_high_risk_assets                                         CAN
top_country_contributor_for_local_critical_infrastructure                            CAN
top_country_contribut

In [90]:
db_global.loc[7328,'overall_percentile_regional_infrastructure_risk']

4

In [101]:
(raw_global_data.groupby(['rcp_scenario', 'physical_impact_year'])['overall_regional_infrastructure_risk'].rank(pct=True, method='max') * 100).loc[[7328,13817,19426,20375,22064]]

7328     4.500628
13817    4.500628
19426    4.500628
20375    4.500628
22064    4.500628
Name: overall_regional_infrastructure_risk, dtype: float64

In [127]:
raw_global_data.to_csv('global_risk_calculated.csv', index= False)